In [1]:
import pandas as pd
import numpy as np

import pickle as pk
import xgboost as xgb

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

/home/felipe/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/felipe/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
pd.set_option("display.max_rows", 8)

In [3]:
df = pd.read_pickle("../../data/processed/df_20features_redshift.pkl")

In [6]:
df

,desg,desi,desr,desz,Xaxis,desg_GP,desi_GP,desr_GP,desz_GP,wavelets,...,f14,f15,f16,f17,f18,f19,f20,type,type_bool,REDSHIFT_SPEC
ID,,,,,,,,,,,,,,,,,,,,,
SN000017,"[[6.984000000004016, 6.122, 6.377], [7.9920000...","[[0.012000000002444722, 0.6447, 49.95], [7.008...","[[0.0, 4.41, 17.58], [8.00800000000163, 1.002,...","[[4.121000000006461, 1.337, 3.581], [7.0230000...","[0.0, 1.3437474747475018, 2.6874949494950036, ...","([2.1790669622347325, 2.508001685841991, 2.869...","([10.397060897840504, 10.577637884754534, 10.7...","([1.5588534229682, 1.5163625398617389, 1.45121...","([2.4493108381108755, 2.6225647004586676, 2.75...","[2.5170703344071526, 3.609106466610777, 4.6966...",...,73.889818,106.164333,90.423999,14.934331,-38.870990,38.843075,-28.337797,II,False,0.39811
SN000019,"[[0.0, 1.325, 5.225], [1.0120000000024447, 10....","[[10.023000000001048, 1.944, 4.071], [23.06599...","[[0.012000000002444722, 2.468, 3.577], [23.050...","[[9.00800000000163, 6.876, 4.915], [14.0159999...","[0.0, 0.9801111111111381, 1.9602222222222763, ...","([2.219675101833163, 2.219675198412055, 2.2196...","([-0.19220466685970194, -0.0681479629423194, 0...","([1.861237782410762, 1.7895540003134789, 1.726...","([3.654910834866328, 3.8235492654180803, 3.993...","[4.43934546360616, 4.439347606905569, 4.439349...",...,-37.724978,-106.674898,6.508267,181.072708,35.825003,-29.788784,52.454903,II,False,0.97361
SN000126,"[[0.0, 0.5652, 2.636], [23.91399999999703, 0.5...","[[7.932999999997264, 1.535, 4.162], [36.811999...","[[15.930000000000291, 1.963, 3.215], [27.02300...","[[7.956999999994878, 0.1151, 2.711], [15.97599...","[0.0, 1.584242424242389, 3.168484848484778, 4....","([0.35689014312282547, 0.36868698625858665, 0....","([0.34290692312134574, 0.39300951014427277, 0....","([0.11936109021103114, 0.23793544213573692, 0....","([-0.6661782956213349, -0.5705737568129523, -0...","[2.6824500442573638, 1.7791579281136436, 1.110...",...,4.312964,17.417031,13.092084,1.662298,-6.498159,14.552209,25.124219,II,False,0.56328
SN000172,"[[0.0, 8.814, 4.79], [2.0, 9.327, 2.25], [9.98...","[[0.031000000002677552, 3.05, 5.436], [2.15600...","[[10.0, 1.734, 1.254], [11.988000000004831, 12...","[[10.039000000004307, 12.9, 2.19], [12.0310000...","[0.0, 1.2923333333333222, 2.5846666666666445, ...","([6.562159405329549, 9.1252195867519, 11.89292...","([2.6439974671168613, 3.295315393791398, 3.956...","([-12.447889370713835, -12.656461305309037, -1...","([10.889899390805923, 11.500413415470769, 12.1...","[6.47820341295315, 11.499857208884713, 17.4609...",...,-26.431645,15.255596,20.829588,18.986775,-19.481710,61.150794,-33.403607,II,False,0.43733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SN899766,"[[6.984000000004016, 2.619, 6.376], [21.973000...","[[0.012000000002444722, 48.86, 49.95], [8.0230...","[[0.0, 19.65, 17.59], [6.992000000005646, 5.34...","[[4.121000000006461, 1.07, 3.581], [7.02300000...","[0.0, 1.474707070707136, 2.949414141414272, 4....","([3.1774853444736557, 3.1774855699966595, 3.17...","([6.273860823260804, 6.090278522008031, 5.9044...","([8.309936227222874, 8.154384056314196, 7.9766...","([1.0347047221511048, 1.209742608349444, 1.388...","[6.3549756079875195, 6.354973557479621, 6.3549...",...,-22.871439,-85.058740,23.367325,86.632538,21.008687,-14.510971,64.077706,II,False,0.84069
SN899772,"[[0.0, 8.811, 25.57], [5.875, 1.341, 4.764], [...","[[0.8550000000032014, 10.26, 30.95], [21.81600...","[[0.015000000006693881, 7.061, 4.154], [32.820...","[[19.851000000002387, 3.151, 4.938], [27.85100...","[0.0, 1.6130000000000542, 3.2260000000001083, ...","([3.375523849152177, 2.9073715158475206, 2.313...","([5.9244796371948585, 5.563986512566748, 5.176...","([6.3162013263673575, 5.386246481590906, 4.336...","([11.990492759478087, 11.552298169626269, 11.0...","[3.341657653722518, 4.5430537588317765, 5.0680...",...,-66.440696,55.496760,-63.973373,-119.170704,23.982747,-77.691256,-40.448599,II,False,0.47005
SN899777,"[[0.0, 3.066

In [7]:
splits = int(len(df)/1100)

X_real = df.loc[:,'f1':'f20']
y_real = np.array(df.REDSHIFT_SPEC)

kf_real = KFold(n_splits = splits)
X_true_test = []
X_to_train = []
y_true_test = []
y_to_train = []
for train_index_real, test_index_real in kf_real.split(X_real):
    print("X_true_test:", train_index_real, "X_to_train:", test_index_real)
    X_true_test.append(X_real.iloc[train_index_real])
    X_to_train.append(X_real.iloc[test_index_real])
    y_true_test.append(y_real[train_index_real])
    y_to_train.append(y_real[test_index_real])

X_true_test: [ 1122  1123  1124 ... 21314 21315 21316] X_to_train: [   0    1    2 ... 1119 1120 1121]
X_true_test: [    0     1     2 ... 21314 21315 21316] X_to_train: [1122 1123 1124 ... 2241 2242 2243]
X_true_test: [    0     1     2 ... 21314 21315 21316] X_to_train: [2244 2245 2246 ... 3363 3364 3365]
X_true_test: [    0     1     2 ... 21314 21315 21316] X_to_train: [3366 3367 3368 ... 4485 4486 4487]
X_true_test: [    0     1     2 ... 21314 21315 21316] X_to_train: [4488 4489 4490 ... 5607 5608 5609]
X_true_test: [    0     1     2 ... 21314 21315 21316] X_to_train: [5610 5611 5612 ... 6729 6730 6731]
X_true_test: [    0     1     2 ... 21314 21315 21316] X_to_train: [6732 6733 6734 ... 7851 7852 7853]
X_true_test: [    0     1     2 ... 21314 21315 21316] X_to_train: [7854 7855 7856 ... 8973 8974 8975]
X_true_test: [    0     1     2 ... 21314 21315 21316] X_to_train: [ 8976  8977  8978 ... 10095 10096 10097]
X_true_test: [    0     1     2 ... 21314 21315 21316] X_to_train: 

# Testing for [0][0]

In [8]:
X_ = X_to_train[0]
y_ = y_to_train[0]

In [9]:
data_dmatrix = xgb.DMatrix(data=X_,label=y_)

In [10]:
kf = KFold(n_splits = 5)
X_train = []
X_test = []
y_train = []
y_test = []
for train_index, test_index in kf.split(X_):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train.append(X_.iloc[train_index])
    X_test.append(X_.iloc[test_index])
    y_train.append(y_[train_index])
    y_test.append(y_[test_index])

In [11]:
#X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=123)

In [12]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [13]:
xg_reg.fit(X_train[0],y_train[0])

[18:09:43] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=0, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=10, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [14]:
preds = xg_reg.predict(X_test[0])

In [15]:
rmse = np.sqrt(mean_squared_error(y_test[0], preds))
print("RMSE: %f" % (rmse))

RMSE: 0.218551


In [16]:
np.round((abs(y_test[0]-preds)/y_test[0])*100)

array([ 29.,  34.,  17.,  46.,  28.,  12.,  11.,  15.,  18.,  86.,  39.,
        26.,   5.,  97.,  20.,  17.,  12., 120.,   2.,  14.,  26.,  63.,
        42.,  25.,  13.,  40.,   7.,  23.,  23.,  26.,  14.,  60.,   8.,
         8.,  81.,  33.,   1.,  11.,  13.,  35.,  78.,  24.,  20.,  37.,
        36.,  40.,  26.,  15.,  55.,  28.,   2.,  22.,  25.,  29.,  17.,
        14.,  36.,   1.,  33.,  34.,   7.,  18.,  39.,  63.,  42.,  30.,
         2.,  18.,  44.,  22.,  26.,  38.,  12.,   9.,  29.,  41.,  17.,
        36.,  59.,  11.,  78.,  38.,  14.,  35.,  36.,  10.,  36.,   8.,
        23.,  34.,  38.,  44.,  29.,  38.,  45.,   5.,  36.,  22.,  35.,
        34.,  26.,  35.,  11.,  37.,  24.,   2.,  26.,  31.,  36.,  77.,
         6.,  12.,  79.,  22.,   9.,  41.,  17.,  67.,  46.,  21.,  15.,
        34.,   0., 639.,   6.,  32.,  30.,  36.,  35.,  41.,  33.,  52.,
        70., 354.,  35.,   8.,  19.,  33.,  25.,   4.,  15.,  42.,  36.,
        13.,  38.,  37.,  26.,   6.,   5.,  80.,  2

In [17]:
true_preds = xg_reg.predict(X_true_test[0])
rmse = np.sqrt(mean_squared_error(y_true_test[0], true_preds))
print("RMSE: %f" % (rmse))

RMSE: 0.217004


# Testing for [12][2]

In [18]:
X_ = X_to_train[12]
y_ = y_to_train[12]

In [19]:
data_dmatrix = xgb.DMatrix(data=X_,label=y_)

In [20]:
kf = KFold(n_splits = 5)
X_train = []
X_test = []
y_train = []
y_test = []
for train_index, test_index in kf.split(X_):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train.append(X_.iloc[train_index])
    X_test.append(X_.iloc[test_index])
    y_train.append(y_[train_index])
    y_test.append(y_[test_index])

In [21]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)

In [22]:
xg_reg.fit(X_train[2],y_train[2])

[18:09:45] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(alpha=10, base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.1, max_delta_step=0, max_depth=5,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=10, n_jobs=0, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=10, reg_lambda=1,
             scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [23]:
preds = xg_reg.predict(X_test[2])

In [24]:
rmse = np.sqrt(mean_squared_error(y_test[2], preds))
print("RMSE: %f" % (rmse))

RMSE: 0.230032


In [25]:
np.round((abs(y_test[2]-preds)/y_test[2])*100)

array([3.000e+01, 2.000e+01, 2.100e+01, 4.100e+01, 1.975e+03, 1.300e+01,
       1.000e+01, 3.100e+01, 9.000e+00, 1.500e+01, 5.000e+00, 2.000e+01,
       7.000e+00, 3.000e+00, 2.000e+00, 1.340e+02, 4.600e+01, 4.000e+00,
       1.000e+01, 3.900e+01, 3.400e+01, 1.300e+01, 3.900e+01, 1.040e+02,
       3.700e+01, 3.100e+01, 3.800e+01, 3.000e+01, 3.400e+01, 2.000e+01,
       3.800e+01, 1.000e+00, 2.700e+01, 2.300e+01, 3.600e+01, 1.400e+01,
       6.000e+00, 4.300e+01, 3.200e+01, 1.600e+01, 9.000e+00, 2.800e+01,
       3.300e+01, 4.600e+01, 1.100e+01, 2.700e+01, 2.100e+01, 3.300e+01,
       3.200e+01, 1.900e+01, 7.000e+00, 4.000e+01, 1.500e+01, 4.400e+01,
       6.000e+00, 1.500e+01, 1.700e+01, 7.000e+00, 0.000e+00, 3.300e+01,
       9.000e+00, 1.030e+02, 2.700e+01, 6.400e+01, 1.000e+01, 3.700e+01,
       2.200e+01, 1.800e+01, 6.000e+00, 3.500e+01, 3.000e+00, 1.100e+01,
       4.000e+00, 2.800e+01, 3.000e+01, 9.000e+00, 8.000e+00, 2.900e+01,
       2.600e+01, 2.500e+01, 6.000e+00, 3.600e+01, 

In [26]:
true_preds = xg_reg.predict(X_true_test[12])
rmse = np.sqrt(mean_squared_error(y_true_test[12], true_preds))
print("RMSE: %f" % (rmse))

RMSE: 0.218714


# K-Folding for [1][1]

In [27]:
X_ = X_to_train[1]
y_ = y_to_train[1]

In [28]:
data_dmatrix = xgb.DMatrix(data=X_,label=y_)

In [29]:
params = {"objective":"reg:linear",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=5,
                    num_boost_round=50,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)


[18:09:47] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[18:09:47] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[18:09:47] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[18:09:47] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.
[18:09:47] WARNING: /workspace/src/objective/regression_obj.cu:167: reg:linear is now deprecated in favor of reg:squarederror.


In [30]:
cv_results

,train-rmse-mean,train-rmse-std,test-rmse-mean,test-rmse-std
0,0.276130,0.003267,0.276512,0.015084
1,0.264389,0.003047,0.265216,0.014583
2,0.254492,0.002626,0.256025,0.014476
3,0.245580,0.002592,0.247628,0.013598
...,...,...,...,...
46,0.180475,0.001117,0.193502,0.005277
47,0.180421,0.001196,0.193453,0.005249
48,0.180421,0.001196,0.193453,0.005249
49,0.180421,0.001196,0.193453,0.005249


In [31]:
print((cv_results["test-rmse-mean"]).tail(1))

49    0.193453
Name: test-rmse-mean, dtype: float64


In [32]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10, n_estimators = 10)